In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p
from extractor.tiingo_extractor import TiingoExtractor
from extractor.fred_extractor import FREDExtractor

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")

In [4]:
analysis = []
market.connect()
periods = ["year","quarter","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test = p.column_date_processing(test)
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["rolling_100"] = test["adjclose"].rolling(window=100).mean()
        test["window_10"] = test["adjclose"].shift(10)
        ep = test.tail(1)["adjclose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjclose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjclose"].item())/test.tail(2).iloc[0]["adjclose"].item()
        report["window_gain"] = (test.tail(2).iloc[0]["window_10"].item() - ep)/ep
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:34<00:00, 14.23it/s]


In [5]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]],on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,ticker,Security,GICS Sector,GICS Sub-Industry
19,0.815103,0.118169,0.143319,0.133765,-0.071916,ALGN,Align Technology,Health Care,Health Care Supplies
431,0.089404,0.124633,0.125625,0.118713,-0.105071,TXT,Textron,Industrials,Aerospace & Defense
270,0.700220,0.078281,0.098462,0.092750,-0.113042,LRCX,Lam Research,Information Technology,Semiconductor Equipment
395,1.251694,0.062791,0.065579,0.087232,-0.094548,RCL,Royal Caribbean Group,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
400,0.211460,-0.000482,0.048414,0.072106,-0.008528,STX,Seagate Technology,Information Technology,"Technology Hardware, Storage & Peripherals"
475,0.326966,0.082316,0.070915,0.072016,-0.078215,WDC,Western Digital,Information Technology,"Technology Hardware, Storage & Peripherals"
117,0.284740,0.085967,0.047586,0.056863,-0.066373,CMCSA,Comcast,Communication Services,Cable & Satellite
304,0.413730,0.112207,0.085370,0.054764,-0.102434,MU,Micron Technology,Information Technology,Semiconductors
265,0.285181,-0.005095,0.046358,0.053511,-0.033751,KLAC,KLA Corporation,Information Technology,Semiconductor Equipment
45,0.514361,0.004606,0.056306,0.051216,-0.049610,AMAT,Applied Materials,Information Technology,Semiconductor Equipment


In [6]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,window_gain
GICS Sector,,,,,
Energy,0.054585,0.083972,0.000084,-0.007838,-0.030207
Financials,-0.001117,0.053145,-0.008257,-0.012002,-0.031636
Materials,0.064395,0.034173,0.014762,-0.003512,-0.024810
Utilities,-0.012861,0.023396,-0.014283,-0.018747,-0.006080
Information Technology,0.246215,0.023044,0.001935,-0.000677,-0.005023
Health Care,0.035693,0.015364,-0.012996,-0.010414,-0.009515
Consumer Discretionary,0.188609,0.015105,-0.002988,-0.008170,0.009126
Real Estate,0.038531,0.011450,-0.022661,-0.021416,0.006032
Industrials,0.164890,0.010192,-0.007951,-0.007723,0.007871


In [7]:
a = pd.DataFrame(analysis)
a.sort_values("window_gain",ascending=False).head(10)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,ticker
282,-0.681564,-0.240000,-0.065574,-0.055249,0.192982,LUMN
16,0.118121,-0.113743,-0.116070,-0.015044,0.187739,ALK
17,-0.038071,-0.101675,-0.023008,-0.019379,0.182034,ALB
411,0.023494,-0.108531,-0.112366,-0.089355,0.167474,LUV
244,-0.001427,-0.141207,0.025130,0.019506,0.156353,IPG
342,0.005636,-0.138342,-0.009026,-0.006875,0.156060,OMC
145,0.080013,-0.117792,-0.042815,-0.021692,0.143406,DFS
171,0.024901,-0.114651,-0.017865,-0.003145,0.142611,EFX
248,-0.104762,-0.057210,-0.100780,-0.017640,0.139319,IVZ
383,-0.150480,-0.134363,-0.122049,-0.013094,0.136785,RTX
